In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
tf.device('/device:GPU:0')
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
EPOCHS=10

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images = 1./255 * train_images
test_images = 1./255 * test_images

170500096/170498071 [==============================] - 6s 0us/step


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import numpy as np

train_sorted = [train_images[train_labels == i, :] for i in range(len(class_names))]
test_sorted = [test_images[test_labels == i, :] for i in range(len(class_names))]

In [33]:
!rm /content/drive/MyDrive/training_checkpoints/*

In [2]:
import numpy as np

def introduce_confusion_on_one_class(train_labels, class_to_error, cur_error_rate=1):
    train_labels = train_labels.flatten()
    should_error = (np.random.rand(train_labels.shape[0]) < cur_error_rate)
    should_error =  np.logical_and(should_error, train_labels == class_to_error)
    random_labels = np.random.randint(10, size=train_labels.shape[0])
    return np.choose(should_error, [train_labels.flatten(), random_labels])
# new_labels = introduce_confusion_on_one_class(train_labels, 1)
# plt.hist(new_labels, bins = 20)
# plt.show()

def introduce_confusion_between_two_classes(train_labels, c1, c2, cur_error_rate=0.5):
    train_labels = train_labels.flatten()
    should_error = (np.random.rand(train_labels.shape[0]) < cur_error_rate)
    flip_classes = np.logical_or(train_labels == c1, train_labels == c2)
    should_error = np.logical_and(should_error, flip_classes)
    flipped = (train_labels == c2) * c1 + (train_labels == c1) * c2
    return np.choose(should_error, [train_labels.flatten(), flipped])

# new_labels = introduce_confusion_between_two_classes(train_labels, 1, 2)
# plt.hist(new_labels, bins = 20)
# plt.show()
#print(new_labels[:10])

def build_and_compile_new_model(checkpoint_location):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='/content/drive/MyDrive/training_checkpoints/'+checkpoint_location,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model, model_checkpoint_callback


In [4]:
import os
touch_prefix = '/content/drive/MyDrive/training_checkpoints/touch'
def touch(point):
    with open(touch_prefix+point, 'a'):
        os.utime(touch_prefix+point, None)

def exists(point):
    return os.path.exists(touch_prefix+point)

def train_experiment(name, train_labels, silent=False, load=False):
    model, model_checkpoint_callback = build_and_compile_new_model(name)

    if not exists(name):
        model.fit(train_images, train_labels, epochs=EPOCHS, 
             validation_data=(test_images, test_labels), 
             callbacks=[model_checkpoint_callback], verbose=(not silent) * 1)

    if load:
        model.load_weights('/content/drive/MyDrive/training_checkpoints/'+name)
    touch(name)

In [6]:
import tqdm

for i in tqdm.tqdm(range(20)):
    train_experiment(f'checkpoint_correct_{i}', train_labels, silent=True)

100%|██████████| 20/20 [16:06<00:00, 48.33s/it]


In [27]:
import itertools, tqdm

#fucked up model for all combos
for fst, snd in tqdm.tqdm(list(itertools.combinations(class_names, 2))):
    first_class = class_names.index(fst)
    second_class = class_names.index(snd)
    confused_train_labels = introduce_confusion_between_two_classes(train_labels, first_class, second_class)
    train_experiment(f'checkpoint_{fst}_{snd}', confused_train_labels, silent=True, load=False)

 13%|█▎        | 2/15 [01:36<10:25, 48.11s/it]

100%|██████████| 15/15 [12:08<00:00, 48.58s/it]


In [39]:
import itertools, tqdm

#fucked up model for all combos
for fst, snd in tqdm.tqdm(list(itertools.combinations(class_names, 2))):
    for i in range(5, 10):
        first_class = class_names.index(fst)
        second_class = class_names.index(snd)
        confused_train_labels = introduce_confusion_between_two_classes(train_labels, first_class, second_class)
        train_experiment(f'checkpoint_{fst}_{snd}_{i}', confused_train_labels, silent=True, load=False)


100%|██████████| 45/45 [3:26:45<00:00, 275.68s/it]


In [6]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

eep = build_and_compile_new_model('checkpoint_correct')
eep[0].summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)               